In [36]:
from public_data.models import Departement, Ocsge, Region, OcsgeDiff, Commune, Cerema, Scot
import pandas as pd
from public_data.storages import DataStorage
from datetime import datetime
from django.contrib.gis.db.models import Union
from tqdm.notebook import tqdm

In [2]:
dept_list = {d.name: d for d in Departement.objects.all()}
region_list = {r.name: r for r in Region.objects.all()} | {
    "Bourgogne-Franche-Comte": Region.objects.get(
        name="Bourgogne-Franche-Comté"
    ),
    "Ile-de-France": Region.objects.get(name="Île-de-France"),
    "Auvergne-Rhone-Alpes": Region.objects.get(name="Auvergne-Rhône-Alpes"),
    "Provence-Alpes-Cote d'Azur": Region.objects.get(
        name="Provence-Alpes-Côte d'Azur"
    ),
}

In [3]:
with DataStorage().open('scote_et_communes.xlsx') as file_stream:
    df_scot = pd.read_excel(file_stream, sheet_name="Liste des SCoT")
    df_city = pd.read_excel(file_stream, sheet_name="Communes Scot")

In [4]:
df_scot

,Code région,Région,Code département,Département,interdépartemental (Oui/non),IDSCoT\n,SCoT,Code Etat simplifié,Libellé Etat simplifié,Code Etat détaillé,Libellé Etat détaillé\n,publication du Pèrimetre,engagement,Arrêt de projet,approbation,Fin échéance,Intégration disposition loi ENE,Type,Nom,Siren
0,1,Guadeloupe,971,Guadeloupe,Non,10386,SCOT DE CAP EXELLENCE,5,SCoT en élaboration,53,SCoT en élaboration : délibération prise,2014-06-30,2015-02-13,NaT,NaT,NaT,NaN,Communauté d'agglomération,Cap excellence,200018653.0
1,1,Guadeloupe,971,Guadeloupe,Non,10484,SCOT DU NORD GRANDE TERRE,5,SCoT en élaboration,53,SCoT en élaboration : délibération prise,2019-06-18,2019-06-19,NaT,NaT,NaT,NaN,Communauté d'agglomération,Du nord grande terre,200044691.0
2,2,Martinique,972,Martinique,Non,416,SCOT DE LA CACEM,30,SCoT approuvé,302,SCoT approuvé,2002-03-22,2003-03-28,2015-07-22,2016-11-16,NaT,Oui,Communauté d'agglomération,Du centre de la martinique,249720061.0
3,2,Martinique,972,Martinique,Non,4147,SCOT DE LA CAESM,30,SCoT approuvé,302,SCoT approuvé,2008-12-03,2010-06-02,2015-11-27,2018-09-25,2024-09-25,NaN,Communauté d'agglomération,De l'espace sud de la martinique,249720053.0
4,2,Martinique,972,Martinique,Non,10252,SCOT CAP NORD MARTINIQUE,20,SCoT en révision,203,SCoT en révision : délibération prise,NaT,2014-03-07,NaT,NaT,NaT,Oui,Communauté d'agglomération,Du pays nord martinique,200041788.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,93,Provence-Alpes-Cote d'Azur,84,Vaucluse,Oui,575,SCOT DU BASSIN DE VIE D'AVIGNON,20,SCoT en révision,204,SCoT en révision : projet arrêté,NaT,2013-07-09,2019-12-09,NaT,2023-12-27,NaN,Syndicat mixte,Scot du bassin de vie d'avignon,258403153.0
454,93,Provence-Alpes-Cote d'Azur,84,Vaucluse,Oui,1120,SCOT DE L ARC COMTAT VENTOUX,30,SCoT approuvé,302,SCoT approuvé,NaT,2014-02-13,2019-03-05,2020-10-09,2026-10-09,Oui,Syndicat mixte,Scot du comtat-ventoux,258403195.0
455,93,Provence-Alpes-Cote d'Azur,84,Vaucluse,Oui,1121,SCOT VAISON VENTOUX,20,SCoT en révision,204,SCoT en révision : projet arrêté,NaT,2015-02-02,2019-11-27,NaT,2022-01-25,NaN,Communauté de communes,Pays vaison ventoux,248400335.0
456,93,Provence-Alpes-Cote d'Azur,84,Vaucluse,Oui,6107,SCOT DU PAYS D'APT,30,SCoT approuvé,302,SCoT approuvé,2002-06-12,2015-07-09,2018-09-06,2019-07-11,2025-07-11,Oui,Communauté de communes,Pays d'apt-luberon,200040624.0


In [8]:
scot_id_list = []
for index, row in df_scot.iterrows():
    if row["Région"] not in region_list:
        # ignore DOMTOM
        continue
    try:
        scot = Scot.objects.get(id=row["IDSCoT\n"])
    except Scot.DoesNotExist:
        scot = Scot(id=row["IDSCoT\n"])
    scot.name = row["Nom"]
    scot.region = region_list[row["Région"]]
    scot.departement = dept_list[row["Département"]]
    scot.is_inter_departement = row["interdépartemental (Oui/non)"] == "Oui"
    scot.state_statut = row["Libellé Etat simplifié"]
    scot.detailed_state_statut = row["Libellé Etat détaillé\n"]
    scot.date_published_perimeter = row["publication du Pèrimetre"]
    scot.date_acting = row["engagement"]
    scot.date_stop = row["Arrêt de projet"]
    scot.date_validation = row["approbation"]
    scot.date_end = row["Fin échéance"]
    scot.is_ene_law = row["Intégration disposition loi ENE"] == "Oui"
    scot.scot_type = row["Type"]
    scot.siren = row["Siren"]
    # scot.save()
    scot_id_list.append(scot.id)
    print(f"Done {scot.name}")

Done Du grand paris
Done Paris - vallee de la marne
Done Marne et gondoire
Done Coulommiers pays de brie
Done Smep du grand provinois
Done Smep nemours gatinais
Done Smep seine et loing
Done Smepa marne ourcq
Done Melun val de seine
Done Rambouillet territoires
Done Gally-mauldre
Done Saint germain boucles de seine
Done Du pays de limours
Done Coeur d'essonne agglomeration
Done Du val d'essonne
Done Grand paris sud seine essonne senart
Done Du grand paris
Done De cergy-pontoise
Done Roissy pays de france
Done Sirdab - scot agglomeration berruyere
Done Syndicat mixte de developpement du pays berry st amandois
Done Syndicat mixte du pays sancerre sologne
Done Syndicat mixte pays de loire val d'aubois
Done Chartres metropole
Done Entre beauce et perche
Done Des portes eureliennes d ile de france
Done Du pays dunois
Done Coeur de beauce
Done Du perche d'eure-et-loir
Done Agglo du pays de dreux
Done Du pays castelroussin - val de l'indre
Done Scot brenne marche
Done Pays de valencay en berr

Done Pays de balagne


In [6]:
df_city

,id,SCoT,insee,Commune
0,2,SCOT BOURG-BRESSE-REVERMONT,01024,Attignat
1,2,SCOT BOURG-BRESSE-REVERMONT,01029,Beaupont
2,2,SCOT BOURG-BRESSE-REVERMONT,01038,Bény
3,2,SCOT BOURG-BRESSE-REVERMONT,01040,Béréziat
4,2,SCOT BOURG-BRESSE-REVERMONT,01053,Bourg-en-Bresse
...,...,...,...,...
29202,10503,SCOT CASA,06128,Saint-Paul-de-Vence
29203,10503,SCOT CASA,06148,Tourrettes-sur-Loup
29204,10503,SCOT CASA,06152,Valbonne
29205,10503,SCOT CASA,06155,Vallauris


In [33]:
unkown_insee_list = []
for index, row in tqdm(df_city.iterrows(), total=len(df_city.index)):
    try:
        if row["id"] in scot_id_list:
            city = Commune.objects.get(insee=row["insee"])
            city.scot_id = row["id"]
            # city.save()
    except Commune.DoesNotExist:
        unkown_insee_list.append(row['insee'])
        print(row['SCoT'], row['insee'], row['Commune'])

  0%|          | 0/29207 [00:00<?, ?it/s]

SCOT DES RIVES DU RHONE 26219 Mureils
SCOT DU PAYS DE BREST 29084 Île-Molène
SCOT SEINE EURE FORET DE BORD 27676 Trois Lacs (Les)
SCOT VAL DE L'AISNE 02695 Saint-Thibaut
SCOT DU PAYS DE PONTIVY 56049 Croixanvec
SCOT DES COEVRONS 53239 Saint-Martin-de-Connée
SCOT DES COEVRONS 53274 Vimarcé
SCOT DU PAYS BARROIS 55138 Culey
SCOT DU COGNACAIS 16010 Ambleville
SCOT DU COGNACAIS 16351 Saint-Simeux
SCOT SUD-VENDEE LITTORAL 85307 Faute-sur-Mer (La)
SCOT DU PAYS DE BRAY 76601 Saint-Lucien
SCOT DU COUSERANS 09304 Suzan
SCOT DU PAYS DE CHAUMONT 52278 Lavilleneuve-au-Roi
SCOT DU PAYS DE LANGRES 52033 Avrecourt
SCOT DU PAYS DE LANGRES 52124 Chézeaux
SCOT DU PAYS DE LANGRES 52465 Saulxures
SCOT DU PAYS HAUTE CORREZE VENTADOUR 19092 Jardin (Le)
SCOT LOUE-LISON 25134 Châtillon-sur-Lison
SCOT LOUE-LISON 25628 Villers-sous-Montrond
SCOT DU PERIGORD NOIR 24089 Cazoulès
SCOT DU PERIGORD NOIR 24314 Orliaguet


In [12]:
Commune.objects.get(insee="52278")

DoesNotExist: Commune matching query does not exist.

In [14]:
Cerema.objects.filter(city_insee="26219")

<QuerySet []>

In [39]:
qs = Commune.objects.values("scot__id").annotate(union_mpoly=Union("mpoly")).order_by("scot__id")
qs

<QuerySet [{'scot__id': None, 'union_mpoly': <MultiPolygon object at 0x7fb02eb92810>}]>

In [41]:
import requests
from requests.auth import HTTPBasicAuth

key = "u6owl86M3UtOTGV8DuaIaG3zEV8a"
secret = "4KM4rkY4M08LcRo9r7C196Ywb1Ia"

response = requests.post(
    "https://api.insee.fr/token",
    auth=HTTPBasicAuth(key, secret),
    data={"grant_type": "client_credentials"}
)
access_token = response.json()['access_token']
access_token

'01e73d80-813d-3747-9403-b57e33415610'

In [46]:
headers = {"Authorization": f"Bearer {access_token}"}
url = "https://api.insee.fr/metadonnees/geo/commune/{code}?date=1950-01-01"
response = requests.get(
    url.format(code="26219"),
    headers=headers
)
response.status_code, response.text

(200,
 '{"code":"26219","uri":"http://id.insee.fr/geo/commune/0a217a9d-b79e-4b7d-bb92-193eda1d6ee7","type":"Commune","dateCreation":"1943-01-01","dateSuppression":"2022-01-01","intituleSansArticle":"Mureils","typeArticle":"0","intitule":"Mureils"}')